In [1]:
import pandas as pd
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import re

## 1. Preprocessing Data
Tahapan ini digunakan untuk melakukan preprocessing data, yang terdiri dari:
- load data dari file csv
- menghapus data-data yang duplikat

In [3]:
df = pd.read_csv('../resources/data/question_list.csv')

# Hapus baris kedua (index ke-1)
df = df.drop(index=0).reset_index(drop=True)

# Ubah nama kolom menjadi 'question'
df.columns = ['question']

# Hilangkan tanda kutip dari setiap string
df['question'] = df['question'].str.replace('"', '').str.strip()

# Hapus duplikat berdasarkan versi lowercase dari 'question'
df = df[~df['question'].str.lower().duplicated()].reset_index(drop=True)

df.head()

,question
0,Internet mati nih
1,Harga STB berapa ya?
2,Internet saya putus sambung
3,Mengapa saya tidak dapat mengisi formulir relo...
4,mengapa ketika ingin melakukan pembayaran tida...


## 2. Labeling Data
karena dataset yang diload diatas belum memiliki labal, sehingga pada tahap ini akan melakukan proses labeling sebelum masuk ke tahap training data.

### a. Labeling with Clustering Data
Labeling pertama dilakukan dengan pendekatan clustering.

proses:
- semua data akan di-embedding untuk merubah data ke dalam bentuk vector
- data akan dicluster dengan menggunakan algoritma DBSCAN
- seteleh data tercluster atau tersegmentasi, selanjutnya dilakukan proses pelabelan secara manual berdasarkan data-data yang ada di setiap cluster. 

In [4]:
def clustering(df, eps, min_samples):
    """
    Melakukan proses clustering data
    """
    df = df.copy()
    
    # embedding data
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(df['question'].tolist())

    # DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
    cluster = dbscan.fit_predict(embeddings)
    df['cluster'] = cluster

    return df

In [5]:
def set_label_by_cluster(df:pd.DataFrame, cluster_value:list, label_name:str):
    """
    Mengatur label string dan integer berdasarkan nilai cluster.
    """

    label_map = {
        'information': 0,
        'request': 1,
        'problem': 2
    }
    if label_name not in label_map:
        raise ValueError("Label harus salah satu dari: 'information', 'request', 'problem'")

    # Set nilai label dan label_int berdasarkan cluster
    df.loc[df['cluster'].isin(cluster_value), 'label'] = label_name
    df.loc[df['cluster'].isin(cluster_value), 'label_int'] = label_map[label_name]

    return df

In [6]:
# data clustering
df_clustered = clustering(df, 0.7, 10)
df_clustered.head()

,question,cluster
0,Internet mati nih,-1
1,Harga STB berapa ya?,-1
2,Internet saya putus sambung,0
3,Mengapa saya tidak dapat mengisi formulir relo...,-1
4,mengapa ketika ingin melakukan pembayaran tida...,-1


In [7]:
# set label by cluster
df_clustered = set_label_by_cluster(df_clustered, [0,1,2,3,4], 'information')
df_clustered

,question,cluster,label,label_int
0,Internet mati nih,-1,NaN,NaN
1,Harga STB berapa ya?,-1,NaN,NaN
2,Internet saya putus sambung,0,information,0.0
3,Mengapa saya tidak dapat mengisi formulir relo...,-1,NaN,NaN
4,mengapa ketika ingin melakukan pembayaran tida...,-1,NaN,NaN
...,...,...,...,...
1724,harga modem berapa?,0,information,0.0
1725,kapan mybiznet ada di appstore?,-1,NaN,NaN
1726,Saya tidak bisa melakukan pembayaran melalui B...,3,information,0.0
1727,untuk promo biznet home ada apa saja?,0,information,0.0


In [8]:
df_clustered.to_csv('../resources/data/labeled_question_list_1.csv', index=False)

### b. Labeling with regex
Selanjutnya melakukan labeling manual menggunakan kata kunci dari setiap label dengan menggunakan regex.

Sehingga data-data yang mengandung kata kunci tertentu akan dilabelkan ke label terkait.

In [10]:
df = pd.read_csv('../resources/data/labeled_question_list_1.csv')

##### regex untuk "Information" label
pada label "Information" digunakan kata kunci: bagaimana, apa, cara, apakah, berapa, kapan

In [11]:
keywords = r'\b(bagaimana|apa|cara|apakah|berapa|kapan)\b'
    
# Filter: hanya baris tanpa label dan mengandung kata kunci
mask = df['label'].isna() & df['question'].str.contains(keywords, flags=re.IGNORECASE, regex=True)

# Tetapkan label
df.loc[mask, 'label'] = 'information'
df.loc[mask, 'label_int'] = 0

C:\Users\adity\AppData\Local\Temp\ipykernel_4476\1377876900.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['label'].isna() & df['question'].str.contains(keywords, flags=re.IGNORECASE, regex=True)


In [13]:
df

,question,cluster,label,label_int
0,Internet mati nih,-1,NaN,NaN
1,Harga STB berapa ya?,-1,information,0.0
2,Internet saya putus sambung,0,information,0.0
3,Mengapa saya tidak dapat mengisi formulir relo...,-1,NaN,NaN
4,mengapa ketika ingin melakukan pembayaran tida...,-1,NaN,NaN
...,...,...,...,...
1724,harga modem berapa?,0,information,0.0
1725,kapan mybiznet ada di appstore?,-1,information,0.0
1726,Saya tidak bisa melakukan pembayaran melalui B...,3,information,0.0
1727,untuk promo biznet home ada apa saja?,0,information,0.0


##### regex for "Problem" label
pada label "Problem" digunakan kata kunci: mati, lambat, down, slow, putus

In [14]:
keywords = r'\b(mati|lambat|down|slow|putus)\b'

# Filter: hanya baris tanpa label dan mengandung kata kunci
mask = df['label'].isna() & df['question'].str.contains(keywords, flags=re.IGNORECASE, regex=True)

# Tetapkan label
df.loc[mask, 'label'] = 'problem'
df.loc[mask, 'label_int'] = 2

C:\Users\adity\AppData\Local\Temp\ipykernel_4476\2418420746.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['label'].isna() & df['question'].str.contains(keywords, flags=re.IGNORECASE, regex=True)


In [15]:
df

,question,cluster,label,label_int
0,Internet mati nih,-1,problem,2.0
1,Harga STB berapa ya?,-1,information,0.0
2,Internet saya putus sambung,0,information,0.0
3,Mengapa saya tidak dapat mengisi formulir relo...,-1,NaN,NaN
4,mengapa ketika ingin melakukan pembayaran tida...,-1,NaN,NaN
...,...,...,...,...
1724,harga modem berapa?,0,information,0.0
1725,kapan mybiznet ada di appstore?,-1,information,0.0
1726,Saya tidak bisa melakukan pembayaran melalui B...,3,information,0.0
1727,untuk promo biznet home ada apa saja?,0,information,0.0


##### export data

In [17]:
df.to_csv('../resources/data/labeled_question_list_2.csv', index=False)

### c. Labeling Manual
pada tahapan ini dilakukan labeling manual berdasarkan id data yang telah ditentukan.

In [18]:
df = pd.read_csv('../resources/data/labeled_question_list_2.csv')

In [ ]:
df

In [129]:
id_list = [
    49, 60, 85, 131, 466, 469, 471, 474, 608, 618, 633, 639, 642, 649, 650, 654, 670, 744, 745, 802, 807, 817, 838, 868, 920, 928, 949, 951, 982, 994, 1014, 1035, 1037, 1038, 1041, 1069, 1070, 1117, 1118, 1120, 1128, 1129, 1170, 1176, 1177, 1190, 1191, 1193, 1214, 1220, 1236, 1252, 1257, 1262, 1334, 1336, 1349, 1360, 1383, 1397, 1403, 1452, 1492, 1508, 1516, 1543, 1582, 1589, 1595, 1596, 1608, 1610, 1612, 1615, 1620, 1627, 1658, 1667, 1678, 1713, 1717, 1722
]

df.loc[df.index.isin(id_list), 'label'] = 'request'
df.loc[df.index.isin(id_list), 'label_int'] = 1

In [130]:
df

,question,cluster,label,label_int
0,Internet mati nih,-1,problem,2.0
1,Harga STB berapa ya?,-1,information,0.0
2,Internet saya putus sambung,0,information,0.0
3,Mengapa saya tidak dapat mengisi formulir relo...,-1,NaN,NaN
4,mengapa ketika ingin melakukan pembayaran tida...,-1,NaN,NaN
...,...,...,...,...
1724,harga modem berapa?,0,information,0.0
1725,kapan mybiznet ada di appstore?,-1,information,0.0
1726,Saya tidak bisa melakukan pembayaran melalui B...,3,information,0.0
1727,untuk promo biznet home ada apa saja?,0,information,0.0


In [ ]:
df.to_csv('../resources/data/labeled_question_list_3.csv', index=False)

### d. Lebelling with LLM (CANCELED)
Karena masih terdapat banyak data yang belum terlabel, selanjutnya akan melakukan proses pelabelan dengan bantuan LLM. 

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

API_KEY = "..."
MODEL_NAME = "llama-3.3-70b-versatile"

In [90]:
df = pd.read_csv('../data/labeled_question_list_3.csv')

In [ ]:
# -- load the model
llm = ChatGroq(
    temperature=0, 
    groq_api_key=API_KEY, 
    model_name=MODEL_NAME
)

output_parser=StrOutputParser()

batch_prompt_template = ChatPromptTemplate.from_template("""
Anda adalah classifier pesan pelanggan ISP. 
Klasifikasikan pesan berikut ke salah satu kategori:
- 'information' (untuk permintaan informasi)
- 'request' (untuk permintaan layanan)
- 'problem' (untuk pengaduan masalah)

Aturan:
1. Hanya respon dengan salah satu dari ['information', 'request', 'problem']
2. Jangan tambahkan penjelasan apapun
3. Jika ragu antara 'request' dan 'problem', pilih 'problem'

Contoh:
Pesan: "Berapa harga paket 100Mbps?" → information
Pesan: "Internet saya lambat hari ini" → problem
Pesan: "Saya mau pindah alamat pemasangan" → request

Contoh input:
1. "Berapa harga paket 100Mbps?"
2. "Internet saya lambat"
3. "Minta pindah alamat"

Contoh output dalam format:
information,problem,request

Sekarang klasifikasikan pesan-pesan berikut:
{messages}
""")

chain=batch_prompt_template|llm|output_parser

In [116]:
input_list = [
    "Teman saya pelanggan Biznet kok dapat gratis kesehatan, saya tidak dapat?",
    "Internetnya lambat",
    "saya mau cek quota saya",    
]

formatted_messages = "\n".join(f"{idx+1}. \"{msg}\"" for idx, msg in enumerate(input_list))

response = chain.invoke({'messages':formatted_messages})
print(response)

problem,problem,information


## 3. Data Cleaning
Tahap ini melakukan pembersihan data dari hasil labeling diatas.

In [19]:
df = pd.read_csv('../resources/data/labeled_question_list_3.csv')

In [20]:
# remove unlabeled data
df = df[df.label.notnull()]
df = df.drop('cluster', axis=1)

In [21]:
df

,question,label,label_int
0,Internet mati nih,problem,2.0
1,Harga STB berapa ya?,information,0.0
2,Internet saya putus sambung,information,0.0
6,apakah untuk pergantian paket layanan bisa sek...,information,0.0
7,apakah jika saat proses pergantian paket layan...,information,0.0
...,...,...,...
1724,harga modem berapa?,information,0.0
1725,kapan mybiznet ada di appstore?,information,0.0
1726,Saya tidak bisa melakukan pembayaran melalui B...,information,0.0
1727,untuk promo biznet home ada apa saja?,information,0.0


In [ ]:
df.to_csv('../resources/data/labeled_question_list_all.csv', index=False)